### Deep learning inference

- Detect tree objects from dataset

In [ ]:
from urbantree.setting import Setting

#SETTING = "settings/by_dop80c_1312/deepforest_r1/setting.yaml"
#SETTING = "settings/by_dop80c_1312/deepforest_r2/setting.yaml"
#SETTING = "settings/by_dop80c_1312/deepforest_r3/setting.yaml"
#SETTING = "settings/opendata_luftbild_dop60_1312/deepforest_r1/setting.yaml"
#SETTING = "settings/opendata_luftbild_dop60_1312/deepforest_r2/setting.yaml"
SETTING = "settings/opendata_luftbild_dop60_1312/deepforest_r3/setting.yaml"

setting = Setting.load_deepforest_setting(SETTING)
setting['model_inference_config']

- detect tree objects from dataset and the resulting bounding boxes DataFrame are saved in pickle objects.
- The object detection can be performed on different patch sizes and
  no extra nms is run on aggregated results.

In [ ]:
from urbantree.deepforest.detection import infer_images

infer_images(**setting)

- Render tree canopy raster images according to the resulting bounding boxes DataFrame saved in pickle objects.

In [ ]:
from urbantree.deepforest.detection import postprocess_render_images

dataset_img_pattern="*.tiff"

postprocess_render_images(**setting,
                          dataset_img_pattern=dataset_img_pattern)

- calculate difference among two datasets

In [ ]:
from urbantree.deepforest.detection import calc_diff
from urbantree.deepforest.detection import create_bbox_shapefile

# tune inference parameters to increase quality
diff_from_inference_param = {
  'confident_min_bbox_size': 20,
  'confident_min_score': 0.9,
  'morphology_factor': 1,
  'patch': [
    {'patch_size': 1200,
    'patch_overlap': 0.3,
    'iou_threshold': 0.8,
    'score_thresh': 0.95},
    {'patch_size': 800,
    'patch_overlap': 0.3,
    'iou_threshold': 0.8,
    'score_thresh': 0.9},
    {'patch_size': 200,
    'patch_overlap': 0.2,
    'iou_threshold': 0.7,
    'score_thresh': 0.8},
    {'patch_size': 96,
    'patch_overlap': 0.18,
    'iou_threshold': 0.6,
    'score_thresh': 0.8}]}
diff_to_inference_param = {
  'confident_min_bbox_size': 30,
  'confident_min_score': 0.8,
  'morphology_factor': 3,
  'patch': [
    {'patch_size': 1200,
    'patch_overlap': 0.3,
    'iou_threshold': 0.8,
    'score_thresh': 0.2},
    {'patch_size': 800,
    'patch_overlap': 0.3,
    'iou_threshold': 0.8,
    'score_thresh': 0.1},
    {'patch_size': 200,
    'patch_overlap': 0.2,
    'iou_threshold': 0.7,
    'score_thresh': 0.01},
    {'patch_size': 96,
    'patch_overlap': 0.18,
    'iou_threshold': 0.6,
    'score_thresh': 0.01}]}

calc_diff(diff_from_setting_path="settings/opendata_luftbild_dop60_1312/deepforest_r3/setting.yaml",
          diff_to_setting_path="settings/by_dop80c_1312/deepforest_r2/setting.yaml",
          aggregate_iou_threshold=0.4,
          diff_iou_threshold=0.08,
          diff_cover_threshold=0.1,
          diff_from_inference_param=diff_from_inference_param,
          diff_to_inference_param=diff_to_inference_param,
          output_bbox_dir='temp/diff/b',
          output_debug_img_dir='temp/diff/debug',
          concurrency=11) 

create_bbox_shapefile(src_img_dir='aerial_images_resampled/opendata_luftbild_dop60_1312',
                    src_bbox_diff='temp/diff/b/diff',
                    output_shp_path='temp/diff/diff.shp',
                    size_min_threshold=200, 
                    iou_threshold=0.2)

In [ ]:
from urbantree.setting import Setting
from urbantree.deepforest.detection import create_bbox_shapefile

setting = Setting.load_deepforest_setting("settings/opendata_luftbild_dop60_1312/deepforest_r3/setting.yaml")
create_bbox_shapefile(src_img_dir='aerial_images_resampled/opendata_luftbild_dop60_1312',
                    src_bbox_diff='interim/opendata_luftbild_dop60_1312/deepforest_r3/inference/b',
                    output_shp_path='temp/2017.shp',
                    model_inference_config=setting['model_inference_config'],
                    size_min_threshold=200,
                    iou_threshold=0.2,
                    geometry_only=False)

# generate heatmap of tree density

In [3]:
import json
import geopandas as gpd
import pandas as pd
from tqdm import tqdm
import numpy as np
from urbantree.utils import geerate_tile_def_from_feature

TREE_DATA = 'temp/2017.shp'
BOUNDARY = 'contrib/munich/munich.boundary.geojson'
SCALE = 0.98
OUTPUT_PNG = 'temp/diff/heatmap'
OUTPUT_NPY = 'temp/diff/heatmap_data'

# shrunk boundary to avoid edge cases
b = gpd.read_file(BOUNDARY)
b.geometry = b.geometry.scale(xfact=SCALE, yfact=SCALE)
boundary = json.loads(b.geometry.to_json())

In [2]:
# load tree data
trees = gpd.read_file(TREE_DATA).to_crs('EPSG:3857')
print("{n} trees loaded".format(n=len(trees)))
trees.head(n=1)

794853 trees loaded


,xmin,ymin,xmax,ymax,label,score,patch_size,local_xmin,local_ymin,local_xmax,local_ymax,geometry
0,1.296352e+06,6.123523e+06,1.296367e+06,6.123538e+06,Tree,1.0,200,757.0,1226.0,781.0,1250.0,"POLYGON ((1296366.625 6123523.210, 1296352.293..."


In [3]:
from pathlib import Path
import numpy as np
import shapely
from tqdm import tqdm

# TODO: make faster!

NEIGHBORHOOD = 500 # meter 
SIZE = 256 # px
ZOOM = range(15, 16)
CONTINUE = True

# calculate and store density
def calc_density(x, y, z, *bbox, continue_mode=True):
  # output png path
  dest = (Path(OUTPUT_NPY) / str(z) / str(x)).joinpath(str(y) + ".npy")
  if continue_mode and dest.exists():
    return
  bboxes = []
  for iy in range(SIZE):
    ymin = bbox[3] - (iy+1)*(bbox[3]-bbox[1])/SIZE
    ymax = bbox[3] -    iy *(bbox[3]-bbox[1])/SIZE
    if ymax-ymin < NEIGHBORHOOD:
      ext = 0.5*(NEIGHBORHOOD-(ymax-ymin))
      ymax = ymax + int(ext)
      ymin = ymin - int(ext)
    for ix in range(SIZE):
      xmin = bbox[0] +    ix *(bbox[2]-bbox[0])/SIZE 
      xmax = bbox[0] + (ix+1)*(bbox[2]-bbox[0])/SIZE
      if xmax-xmin < NEIGHBORHOOD:
        ext = 0.5*(NEIGHBORHOOD-(xmax-xmin))
        xmax = xmax + ext
        xmin = xmin - ext
      tile_area = (xmax-xmin)*(ymax-ymin)
      bboxes.append([ix, iy, tile_area,
                     shapely.geometry.box(xmin, ymin, xmax, ymax)])    
  bboxes = pd.DataFrame(bboxes, columns=['ix','iy','tile_area','geometry'])
  bboxes = gpd.GeoDataFrame(bboxes, geometry='geometry', crs=3857)
  # find tree ratio for each bbox (PIXEL)
  within = gpd.sjoin(trees, bboxes, predicate='within')
  if len(within):
    within['tree_ratio'] = (within.xmax-within.xmin)*(within.ymax-within.ymin)/within.tile_area
    within = within.groupby(['iy','ix']).apply(lambda x: x.tree_ratio.sum())
    within = within.reset_index(name='tree_ratio')
  # save 
  array = np.zeros([SIZE, SIZE, 1], dtype=np.float32)
  for _, row in within.iterrows():
    array[int(row.iy),int(row.ix)] = row.tree_ratio
  dest.parent.mkdir(parents=True,exist_ok=True)
  np.save(file=dest, arr=array)

# covering tile coordinates
tiles = geerate_tile_def_from_feature(boundary['features'], list(map(lambda x: str(x), ZOOM)))
for [x, y, z, *bbox] in tqdm(list(tiles)):
  calc_density(x, y, z, *bbox, continue_mode=CONTINUE)


100%|██████████| 528/528 [15:07:39<00:00, 103.14s/it]  


In [39]:
from PIL import Image
from matplotlib import cm

#COLORMAP = cm.get_cmap('RdYlGn')
COLORMAP = cm.get_cmap('YlGn', 12)

for npy in Path(OUTPUT_NPY).rglob("*.npy"):
  src = np.load(npy)
  out = Path(OUTPUT_PNG) / npy.parent.parent.name / npy.parent.name / (npy.stem + ".png")
  dest = np.zeros([src.shape[0], src.shape[1], 4], dtype=np.uint8)
  for x,y,z in [(x, y,z) for x, y,z in np.ndindex(src.shape)]:
    color = (np.array(COLORMAP(src[x,y,z].item())[:4]) * 255).astype('uint8')
    dest[x,y] = color
  im = Image.fromarray(dest)
  out.parent.mkdir(parents=True,exist_ok=True)
  im.save(out)
